In [280]:
using Revise
using Bilevel

using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using Plots
using LinearAlgebra
using StaticArrays
using Interpolations

In [198]:
urdf = joinpath("..", "..", "urdf", "little_dog", "little_dog2d.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), SVector([0.,0.,0.]...))
normal = FreeVector3D(default_frame(floor), SVector([0.,0.,1.]...))
floor_obs = Obstacle(floor, point, normal, :xyz, 20.)

obstacles = [floor_obs]
env = Environment(mechanism, urdf, obstacles);

In [9]:
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf));
open(mvis)
# IJuliaCell(mvis)

┌ Info: Serving MeshCat visualizer at http://127.0.0.1:8700
└ @ MeshCat /home/blandry/.julia/packages/MeshCat/WlA0B/src/servers.jl:24


Process(`xdg-open http://127.0.0.1:8700`, ProcessExited(0))

Created new window in existing browser session.


In [369]:
N = 8
Δt = 0.025
q0 = [0., -.275, 0., 0., 0., 0., 0., pi/4, pi/4, -pi/4, -pi/4, -pi/2, -pi/2, pi/2, pi/2] 
v0 = zeros(num_velocities(mechanism))

sim_data = get_sim_data_indirect(mechanism,env,Δt)
null_ctrl! = (u,t,x) -> u[:] .= 0.
x0 = MechanismState(mechanism)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
traj_sim = Bilevel.simulate(sim_data,null_ctrl!,x0,N)
setanimation!(mvis, traj_sim[6], traj_sim[1])

## Indirect Method

In [370]:
q_nom = copy(traj_sim[1][end])
q_min = min.(q_nom, [-5., -.5, -.2*pi, 0., 0., 0., 0., -3.5, -3.5, -3.5, -3.5, -3.1, -3.1, -3.1, -3.1])
q_max = max.(q_nom, [5., -0.1, .2*pi, 0., 0., 0., 0., 2.4, 2.4, 2.4, 2.4, 1., 1., 1., 1.])

x_goal = 0.075

N = 10
Δt = 0.05

function add_prob_constraints!(sim_data)
    vs = sim_data.vs

    add_eq!(sim_data, :cq1, num_positions(mechanism), x -> vs(x, :q1) - q_nom)
    add_eq!(sim_data, :cv1, num_velocities(mechanism), x -> vs(x, :v1))
    add_ineq!(sim_data, :cqNx, 1, x -> x_goal - vs(x, Symbol("q", N))[1])
    add_eq!(sim_data, :cqNlegs, num_positions(mechanism)-1, x -> vs(x, Symbol("q", N))[2:end] - q_nom[2:end])
    
    for n = 1:N
        add_eq!(sim_data, Symbol("cq1", n), 2, x -> vs(x, Symbol("q", n))[8:9] + vs(x, Symbol("q", n))[11:-1:10])
        add_eq!(sim_data, Symbol("cq2", n), 2, x -> vs(x, Symbol("q", n))[12:13] + vs(x, Symbol("q", n))[15:-1:14])
    end
end

function add_prob_obj!(sim_data)
    for n = 1:N-1
        add_obj!(sim_data, Symbol("u", n), x -> sim_data.vs(x, Symbol("u", n))' * sim_data.vs(x, Symbol("u", n)))
    end
end

function get_prob_limits(sim_data)
    x_min = -1e19*ones(sim_data.vs.num_vars)
    x_max = 1e19*ones(sim_data.vs.num_vars)
    
    # joint limits
    add_box_con_snopt!(x_min, x_max, sim_data, :q, q_min, q_max, 1:N)
    add_box_con_snopt!(x_min, x_max, sim_data, :h, [1. * Δt], [1. * Δt], 1:N-1)
    
    # torques limits
    u_limit = 100.
    add_box_con_snopt!(x_min, x_max, sim_data, :u, -u_limit * ones(num_velocities(mechanism)), u_limit * ones(num_velocities(mechanism)), 1:N-1)
    
    x_min, x_max
end

function get_prob_init(sim_data)
    x0 = zeros(sim_data.vs.num_vars)
    for n = 1:N
        qn = copy(q_nom)
        qn[1] = (n/N)*(x_goal - q_nom[1]) + q_nom[1]
        x0[sim_data.vs(Symbol("q", n))] .= qn
    end
    
    x0
end

function setup_prob!(sim_data)
    add_prob_constraints!(sim_data)
    add_prob_obj!(sim_data)
    x_min, x_max = get_prob_limits(sim_data)
    x0 = get_prob_init(sim_data)
    
    x0, x_min, x_max
end

setup_prob! (generic function with 1 method)

In [254]:
sim_data = get_trajopt_data_indirect(mechanism,env,Δt,N,relax_comp=false)
x0, x_min, x_max = setup_prob!(sim_data);

In [258]:
con_tol = 1e-3
obj_tol = 1e-3
max_iter = 50000

traj_indirect = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, verbose=1, opt_tol=obj_tol, major_feas=con_tol, minor_feas=con_tol, max_iter=max_iter);

x0 = traj_indirect[9]

setanimation!(mvis, traj_indirect[7], traj_indirect[1])

Finished successfully: requested accuracy could not be achieved


In [275]:
setanimation!(mvis, traj_indirect[7], traj_indirect[1])

In [313]:
t_step = traj_indirect[7]
q_step = traj_indirect[1]
u_step = traj_indirect[2][2:end];

In [330]:
reps = 1

ttraj = t_step
qtraj = q_step
utraj = u_step
for i = 1:(reps-1)
    ttraj = vcat(ttraj, t_step .+ ttraj[end])
    qtraj = vcat(qtraj, map(q -> q + vcat(qtraj[end][1],zeros(length(q)-1)), q_step))
    utraj = vcat(utraj, u_step)
end

setanimation!(mvis, ttraj, qtraj)

## Semidirect Method

In [396]:
sim_data = get_trajopt_data_semidirect(mechanism,env,Δt,N,relax_comp=false)
x0, x_min, x_max = setup_prob!(sim_data);

In [397]:
con_tol = 1e-3
obj_tol = 1e-3
max_iter = 50000
steps = 20

for n = 1:N-1
    sim_data.fric_options[n]["num_sosteps"] = steps
end

traj_semidirect = Bilevel.trajopt(sim_data, x0=x0, x_min=x_min, x_max=x_max, verbose=1, opt_tol=obj_tol, major_feas=con_tol, minor_feas=con_tol, max_iter=max_iter)

x0 = traj_semidirect[9]

setanimation!(mvis, traj_semidirect[7], traj_semidirect[1])

Terminated after numerical difficulties: singular basis


In [399]:
setanimation!(mvis, traj_semidirect[7], traj_semidirect[1])

In [312]:
t_step = traj_semidirect[7]
q_step = traj_semidirect[1]
u_step = traj_semidirect[2][2:end];

## Simulation

In [349]:
ctrl_itp = Vector()
utraj_mat = hcat(utraj...)
for i = 1:num_velocities(mechanism)
    # TODO support variable step size
    push!(ctrl_itp, Interpolations.CubicSplineInterpolation(0.:Δt:Δt*(N-1)*reps,utraj_mat[i,:],extrapolation_bc=0.))
end

function feedforward_ctrl!(u,t,x)
    u[:] = map(ctrl_itp) do itp
       itp(t) / 100.
    end
end

feedforward_ctrl! (generic function with 1 method)

In [351]:
N = 100
Δt = 0.005

q0 = qtraj[1]
v0 = zeros(num_velocities(mechanism))

sim_data = get_sim_data_indirect(mechanism,env,Δt)
state0 = MechanismState(mechanism)
set_configuration!(state0, q0)
set_velocity!(state0, v0)
setdirty!(state0)

traj_sim_ctrl = Bilevel.simulate(sim_data,feedforward_ctrl!,state0,N);

setanimation!(mvis, traj_sim_ctrl[6], traj_sim_ctrl[1])

In [388]:
maximum(abs.(vcat(traj_semidirect[3]...)))

7.558100786582636